In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hys
import tkinter.filedialog as tkf
import scipy.optimize as spopt
import scipy.fftpack as spfft
from sklearn.linear_model import Lasso

In [ ]:
img_adr = tkf.askopenfilename()
print(img_adr)

In [ ]:
crop_size = 200
img = hys.load(img_adr)
print(img)
sliced = img.data[200:200+crop_size, 200:200+crop_size]
print(sliced.shape)
sy, sx = sliced.shape

In [ ]:
def line_hop(img_shape, hop_bottom, hop_top):
    sampling_matrix = np.zeros(img_shape)
    print(sampling_matrix.shape)
    
    line_chosen = []
    line = 0
    hop_range = range(hop_bottom, hop_top, 1)
    while True:
        hop = np.random.choice(hop_range)
        line += hop
        if line >= img_shape[0]:
            break
        line_chosen.append(line)
    
    for i in line_chosen:
        temp = i
        for j in range(img_shape[1]):
            while(True):
                hop = np.random.choice([-1, 0, 1], p=[0.10, 0.80, 0.10])
                temp += hop
                if temp >= 0 and temp < img_shape[0]:
                    break
            sampling_matrix[temp, j] = 1
    
    
    print(("sampling ratio = %.2f %%")%(100*np.sum(sampling_matrix)/(img_shape[0]*img_shape[1])))
    
    return sampling_matrix

In [ ]:
def dct2d(x):
    return spfft.dct(spfft.dct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

def idct2(x):
    return spfft.idct(spfft.idct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

In [ ]:
phi = np.zeros([sy, sx])
phi[np.nonzero(sliced)] = 1.0

#phi = line_hop([sy, sx], 1, 3)
phi_prime = phi.flatten()
ri = np.where(phi_prime==1)[0]

#sampling_ratio = 0.5
#ri = np.random.choice(sy*sx, np.round(sy*sx*sampling_ratio), replace=False )

In [ ]:
#sampling_ratio = 0.5
#ri = np.random.choice(sy*sx, np.round(sy*sx*sampling_ratio), replace=False )
    
dctrow = spfft.idct(np.identity(sx), norm='ortho', axis=0)
dctcol = spfft.idct(np.identity(sy), norm='ortho', axis=0)
    
A = np.kron(dctrow, dctcol)
A = A[ri, :]
print(A.shape)
print("A matrix complete")
del phi_prime
del dctrow
del dctcol

In [ ]:
b = sliced.T.flat[ri]
b = np.expand_dims(b, axis=1)
print(b.shape)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax[0].imshow(sliced, cmap="afmhot")
ax[0].axis("off")
ax[1].imshow(phi, cmap="afmhot")
ax[1].axis("off")
plt.show()

In [ ]:
lasso = Lasso(alpha=1E-6, max_iter=2000, tol=0.00001, random_state=56)

lasso.fit(A, b)
Xa = idct2(np.array(lasso.coef_).reshape(sy, sx).T)
fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax[0].imshow(sliced, cmap="afmhot")
ax[0].axis("off")
ax[1].imshow(Xa, cmap="afmhot")
ax[1].axis("off")
plt.show()